## Need for Vectorization

Our training procedure implementation in Lecture 1 takes one example at a time (that is the reason for the 'for loop') for processing. In a real-world setting, this is not optimal as is it is time consuming. 

We can vectorize this step to reduce the cost. Here, we convert the scalars into vectors/matrices and use matrix maths to make these operations faster.

Let's look at a concrete example now.

For more details on Numpy, refer this great guide.
https://betterprogramming.pub/numpy-illustrated-the-visual-guide-to-numpy-3b1d4976de1d

In [1]:
# Create a 10K integer list and multiply every number by 2

size = 10000

num_list = range(size)
%timeit num_list_new = [num*2 for num in num_list]

708 µs ± 5.62 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [2]:
import numpy as np

# Convert the list into numpy vector
num_vector = np.array(num_list)

%timeit num_vector * 2

9.73 µs ± 674 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


Once we converted the python list into a numpy array, the same multiplication took fraction of the time taken by the for loop.

Identifying and refactoring such operations into vector based ones is an important skill to acquire. In this case, we will vectorize the loss and gradient functions to make our regression model training more efficient.

https://numpy.org/doc/stable/reference/generated/numpy.dot.html

In [3]:
a = np.array([1,2,3,4])
a.shape

(4,)

In [4]:
b = np.array([2])
b.shape

(1,)

In [5]:
# To perform dot product we need a's last dim should be equal to b's first dim.
a = a.reshape(-1, 1)  # Get the shape of a in order
print(np.dot(a, b))   # Same as a @ b
print(np.dot(a, b).shape)

[2 4 6 8]
(4,)


In [6]:
# This is our toy dataset. Just two python lists
X = [25, 30, 40, 20, 15, 60, 33, 52, 70]  # Age
y = [65, 70, 78, 60, 45, 70, 65, 65, 60]  # Weight in Kg

In [7]:
# Few python package imports

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
# Parameter vector (1 dimensional for now)
w = np.array([1])
def calculate_loss(self, x, y):
    # Vectorized - Taking all examples and do a dot multiply
    wx = np.dot(x, w)
    loss = ((wx - y) **2).mean()
    print(loss)

We have removed the for loop with a vector operation.

In [9]:
'''
This is a vectorized implementation of the simple linear regression model.
'''
class SimpleRegression():
    def __init__(self):
        self.params = None # parameter w
        self.lr = 0.0002 # Learning rate
        self.iterations = 10
    
    def _calculate_loss(self, x, y):
        loss = ((np.dot(x, self.params) - y) **2).mean()
        return loss
    
    def _calculate_gradient(self, x, y):
        residue = np.dot(x, self.params) - y
        grad = np.dot(x.T, residue)
        grad /= len(y)
        return grad.item()
    
    def fit(self, x, targets, iterations=None):
        if not iterations:
            iterations = self.iterations
        self.params = np.ones(x.shape[1])
        for _ in range(iterations):
            loss = self._calculate_loss(x, targets)
            grad = self._calculate_gradient(x, targets)
            self.params += -self.lr * grad
            print(f"Loss: {loss:.2f} Gradient : {grad:.2f} Updated params : {self.params.item()}")
            
        return self.params
    def predict(self, x):
        return np.dot(x, self.params)

In [10]:
data = np.array(X).astype(float).reshape(-1,1)
model = SimpleRegression()
model.fit(data, y)

Loss: 948.56 Gradient : -733.56 Updated params : 1.1467111111111112
Loss: 771.68 Gradient : -472.03 Updated params : 1.2411180809876543
Loss: 698.44 Gradient : -303.75 Updated params : 1.3018679171368779
Loss: 668.12 Gradient : -195.46 Updated params : 1.3409597617007238
Loss: 655.56 Gradient : -125.78 Updated params : 1.3661149293237302
Loss: 650.36 Gradient : -80.94 Updated params : 1.3823020001872723
Loss: 648.21 Gradient : -52.08 Updated params : 1.3927182004316188
Loss: 647.31 Gradient : -33.51 Updated params : 1.3994209095532975
Loss: 646.94 Gradient : -21.57 Updated params : 1.4037340283985518
Loss: 646.79 Gradient : -13.88 Updated params : 1.4065094724519303


array([1.40650947])

In [11]:
# Predict weight for a person with age 28
model.predict([28])  

39.38226522865405

## Broadcasting Intro

[Numpy Broadcasting](https://numpy.org/doc/stable/user/basics.broadcasting.html)

In [12]:
# Create a numpy vector and add a scalar to it.

data_s = 2  # scalar
data_vec = np.array([2, 1, 3, 7, 8])  # vector
data_vec + data_s  # vector + scalar addition

array([ 4,  3,  5,  9, 10])

We just did broadcasting without knowing it. If you closely look at the above addition, numpy allowed us to add a scalar to a vector. i.e, it added every element of the vector by the scalar value. To achieve this simple operation, numpy internally performed 'broadcasting'.

Numpy simply created a vector of the shape 'data_vec' by broadcasting the same value across the dimensions.

data_s was converted to [2, 2, 2, 2, 2]

In [13]:
data_vec.shape

(5,)

In [14]:
# Another example

data_2 = np.array([5, 1, 4])
data_3 = np.array([[1, 2, 3],[4, 5, 6], [7, 8, 9]])
data_3 + data_2

array([[ 6,  3,  7],
       [ 9,  6, 10],
       [12,  9, 13]])

In [15]:
print(data_2.shape)
print(data_3.shape)

(3,)
(3, 3)


In this case, numpy copied the variable data_2 three times (row wise) to make the addition possible. 

In [16]:
data_4 = np.array([[1, 2, 3, 4], [5, 6, 7, 8]])
data_4 + data_2

ValueError: operands could not be broadcast together with shapes (2,4) (3,) 

The above operation fails because numpy doesn't know how to do the broadcasting.

For more details read the excellent explanation provided [here](https://numpy.org/doc/stable/user/basics.broadcasting.html)

This is just an introduction to this very important aspect, we will cover this topic in more detail as we move forward. When we get into training models that involves many dimensions (multivariate data, images), any inadvertant broadcasting is very challenging to debug.